# TITLE : MODELS OF DENSITY FROM COMPOSITIONAL DATA MWC7+  TEMP PRESSURE USING MACHINE LEARNING ALGORITHM

OBJECTIVE : TRINING OF MODELS FOR FUTURE SELECTION FOR PREDICTION OF DENSITY USING WIDE RANGE OF COMPOSITION DATA.

THIS FILE AUTOMATICALLY FIT MODELS AND STORE MODELS AT GIVEN PATH 

IF REVIEWER WANT TO CHECK SIMILLAR MODELS USED TO PREDICT TEST OR NOT WHICH TRAINED HERE THAN IN MODEL VALIDATION FILE OPTIMIZED PARAMETER CAN BE CKECKED WHICH AVOID RETRAINING WHICH TAKE A LOT TIME AS WELL AS TO CHECK DATA TRAIN AND TEST ALREADY SEPRATED AND STAROED INTO DATASOURCE FROM PREPROCESSING FILE SAME DATA USED HERE WHICH VERIFIED BY CHEKING EXCEL FILES 


ALGORITHM APPLIED : SUPPORT VECTOR MACHINE, KNN, XGB , ANN

In [1]:
#DATA EXTRACTION, MANIPULATION, VIZULIZATION LIBRARY
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#STATISTICAL TOOLS LIBRARY
import scipy.stats as stat
import pylab 
from statsmodels.stats.outliers_influence import variance_inflation_factor

#DATA FETURES OPERATION LIBRARY
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler

#MODELING LIBRARY
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

#MODELLING OF DEEP LEARNING MODEL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner.tuners import RandomSearch


#MODEL EVALUATION LIBRARY
from sklearn.metrics import mean_squared_error , mean_absolute_error


#Model saving
import pickle


<ipython-input-1-1ef187d9cf9b>:33: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
path = "C:\\STUDY DRIVE\\DENSITY PAPER THESIS\\DENSITY TREND\\MODEL 3\\DATASOURCE\\"
file_name = "TRAIN.xlsx"
model_path = "C:\\STUDY DRIVE\\DENSITY PAPER THESIS\\DENSITY TREND\\MODEL 3\\MODELS\\"

export_data_path = "C:\\STUDY DRIVE\\DENSITY PAPER THESIS\\DENSITY TREND\\MODEL 3\\EXPORTED DATA\\"

train_raw = pd.read_excel(path+file_name)

In [3]:
train_raw.columns

Index(['Sr.No', 'H2S', 'N2', 'CO2', 'C1', 'C2', 'C3-C6', 'C7+', 'MWC7+',
       'Temp', 'P', 'Density'],
      dtype='object')

In [4]:
train = train_raw[['H2S', 'N2', 'CO2', 'C1', 'C2', 'C3-C6', 'C7+', 'MWC7+',
       'Temp', 'P', 'Density']]

In [5]:
train.head()

,H2S,N2,CO2,C1,C2,C3-C6,C7+,MWC7+,Temp,P,Density
0,0.0,0.0037,0.0000,0.2289,0.0059,0.0636,0.6979,167.0,336.1,89.4,780.1
1,0.0,0.0020,0.0021,0.5155,0.0419,0.1387,0.2998,186.0,367.2,241.4,594.3
2,0.0,0.0044,0.0045,0.3505,0.0464,0.1118,0.4824,225.0,355.6,344.8,785.4
3,0.0,0.0044,0.0045,0.3505,0.0464,0.1118,0.4824,225.0,355.6,206.9,772.0
4,0.0,0.0044,0.0045,0.3505,0.0464,0.1118,0.4824,225.0,355.6,173.8,768.3


In [6]:
X_train = train.drop("Density",axis = 1)
y_train = train.Density

                              #### Scalling Dataset ####

In [7]:
scaler = StandardScaler()
x_train = scaler.fit_transform(X_train)

In [8]:
#following model path follows for all models location
scaler_file = 'scaler.sav'
pickle.dump(scaler , open(model_path+scaler_file , "wb") )

##### .......................................................................................SectionBreak.......................................................................................

## 2. SVR MODEL

                              #### Model tuning for svr Regression ####

In [9]:
svr_model = SVR()

In [10]:
svr_para = {'C':range(4000,10000,100),'gamma':np.arange(0.000,0.002,0.0001)}
svr_grid = GridSearchCV(svr_model,svr_para, cv = 5 , verbose=3, n_jobs=-1)

In [11]:
svr_grid.fit(x_train,y_train)

Fitting 5 folds for each of 1200 candidates, totalling 6000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 5936 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 6000 out of 6000 | elapsed:    6.0s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': range(4000, 10000, 100),
                         'gamma': array([0.    , 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007,
       0.0008, 0.0009, 0.001 , 0.0011, 0.0012, 0.0013, 0.0014, 0.0015,
       0.0016, 0.0017, 0.0018, 0.0019])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [12]:
svr_grid.best_params_

{'C': 9900, 'gamma': 0.0019}

In [13]:
svr_best_para = svr_grid.best_params_

In [14]:
svr_regression = SVR( C = svr_best_para["C"],
                      gamma = svr_best_para["gamma"])
svr_regression.fit(x_train,y_train)

SVR(C=9900, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.0019,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

                                          #### Model saveing #### 

In [15]:
svr_file = 'svr_model.sav'
pickle.dump(svr_regression , open(model_path+svr_file , "wb") )

## 2. KNN Model

In [16]:
knn_model = KNeighborsRegressor()

In [17]:
knn_para = {"n_neighbors"  : range(2,11)}
knn_grid = GridSearchCV(knn_model,knn_para, verbose=3, n_jobs=-1)

In [18]:
knn_grid.fit(x_train,y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  45 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid={'n_neighbors': range(2, 11)}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=3)

In [19]:
knn_grid.best_params_

{'n_neighbors': 3}

In [20]:
knn_best_para = knn_grid.best_params_

In [21]:
knn_regression = KNeighborsRegressor( n_neighbors = knn_best_para["n_neighbors"])

knn_regression.fit(x_train,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                    weights='uniform')

                                          #### Model Saving #### 

In [22]:
knn_file = 'knn_model.sav'
pickle.dump(knn_regression , open(model_path+knn_file , "wb") )

## 3. XGB Model

In [23]:
xgb_model = XGBRegressor()

In [24]:
xgb_para={
   
    'learning_rate': np.arange(0.1,0.2,0.04),
    'max_depth': range(2,10,1),
    'n_estimators':range(90,150,10),
    "gamma" : np.arange(0.1,0.5,0.3),
    "min_child_weight": range(1,10,2)
}

In [25]:
xgb_grid = GridSearchCV(xgb_model,xgb_para, cv = 5 , verbose=3, n_jobs=-1)

In [26]:
xgb_grid.fit(x_train,y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 1440 candidates, totalling 7200 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 480 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 928 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done 1504 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done 2208 tasks      | elapsed:   46.6s
[Parallel(n_jobs=-1)]: Done 3040 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 4000 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 5088 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 6304 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 7200 out of 7200 | elapsed:  2.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estima...
                                    reg_lambda=None, scale_pos_weight=None,
                                    subsample=None, tree_method=None,
                                    validate_parameters=None, verbosity=None),
             iid='deprecated', n_jobs=-1,
             param_grid={

In [27]:
xgb_grid.best_params_

{'gamma': 0.4,
 'learning_rate': 0.1,
 'max_depth': 4,
 'min_child_weight': 7,
 'n_estimators': 140}

In [28]:
xgb_best_para = xgb_grid.best_params_

In [29]:
xgb_regression = XGBRegressor(
                    learning_rate = xgb_best_para["learning_rate"],
                    max_depth = xgb_best_para["max_depth"],
                    n_estimators = xgb_best_para["n_estimators"],
                    gamma = xgb_best_para["gamma"],
                    min_child_weight = xgb_best_para["min_child_weight"]
                    )
xgb_regression.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.4, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=4,
             min_child_weight=7, missing=nan, monotone_constraints='()',
             n_estimators=140, n_jobs=8, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [30]:
xgb_file = 'xgb_model.sav'
pickle.dump(xgb_regression , open(model_path+xgb_file , "wb") )

.......................................THE END..........................................................